In [9]:
!conda install ffmpeg -c conda-forge

Channels:
 - conda-forge
 - defaults
Platform: win-64
Solving environment: ...working... done

# All requested packages already installed.



C:\Users\kalin\miniconda3\Lib\site-packages\conda\base\context.py:969: FutureWarning: Adding 'defaults' to channel list implicitly. is deprecated and will be removed in 25.3. 

To remove this warning, please choose a default channel explicitly via 'conda config --add channels <name>', e.g. 'conda config --add channels defaults'.
  deprecated.topic(


In [8]:
!pip install pyaudio numpy

In [4]:
import subprocess

# Check if FFmpeg is available
try:
    subprocess.run(['ffmpeg', '-version'], capture_output=True, check=True)
    print("FFmpeg is installed and accessible")
except FileNotFoundError:
    print("FFmpeg is not installed or not in PATH")
except subprocess.CalledProcessError as e:
    print(f"Error running FFmpeg: {e}")

FFmpeg is installed and accessible


In [5]:
import os
import soundfile as sf  # You might need to pip install soundfile
import whisper

# Check current working directory
print(f"Current working directory: {os.getcwd()}")

audio_path = os.path.abspath("../data/audio/test.mp3")
print(f"Full path to audio file: {audio_path}")

# More detailed file checking
if os.path.exists(audio_path):
    try:
        # Try to open the audio file first
        data, samplerate = sf.read(audio_path)
        print(f"Successfully loaded audio file. Sample rate: {samplerate}")
        
        # If audio loads successfully, proceed with whisper
        model = whisper.load_model("base")
        result = model.transcribe(audio_path)
        print(f'The text in video: \n {result["text"]}')
    except Exception as e:
        print(f"Error loading audio file: {str(e)}")
else:
    print(f"Error: File not found at path: {audio_path}")

Current working directory: c:\Users\kalin\Work\gaia2\notebooks
Full path to audio file: c:\Users\kalin\Work\gaia2\data\audio\test.mp3
Successfully loaded audio file. Sample rate: 44100


c:\Users\kalin\miniconda3\envs\gaiaenv\lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


The text in video: 
  Your hands lie open in the long fresh grass. The finger points look through like rosy blooms. Your eyes smile peace. The pasture gleams and glooms beneath billowing skies that scatter and amass. All round our nest far as the eye can pass. Our golden king cup feels with silver edge where the cow partially skirts the Hawthorn hedge to his visible silence still is the hourglass. Your hands lie open in the long fresh grass. The finger points look through like rosy blooms. Your eyes smile peace. The pasture gleams and glooms beneath billowing skies that scatter and amass. All round our nest far as the eye can pass. Our golden king cup feels with silver edge where the cow partially skirts the Hawthorn hedge to his visible silence still is the hourglass.


In [6]:
import whisper
model = whisper.load_model("base")
result = model.transcribe("../data/audio/test.mp3")
print(f' The text in video: \n {result["text"]}')

 The text in video: 
  Your hands lie open in the long fresh grass. The finger points look through like rosy blooms. Your eyes smile peace. The pasture gleams and glooms beneath billowing skies that scatter and amass. All round our nest far as the eye can pass. Our golden king cup feels with silver edge where the cow partially skirts the Hawthorn hedge to his visible silence still is the hourglass. Your hands lie open in the long fresh grass. The finger points look through like rosy blooms. Your eyes smile peace. The pasture gleams and glooms beneath billowing skies that scatter and amass. All round our nest far as the eye can pass. Our golden king cup feels with silver edge where the cow partially skirts the Hawthorn hedge to his visible silence still is the hourglass.


Available Audio Devices:
- Index 0: Microsoft Sound Mapper - Input
- Index 1: Microphone Array (Realtek(R) Au
- Index 4: Primary Sound Capture Driver
- Index 5: Microphone Array (Realtek(R) Audio)
- Index 9: Microphone Array (Realtek(R) Audio)
- Index 10: Microphone (Realtek HD Audio Mic input)
- Index 13: PC Speaker (Realtek HD Audio 2nd output with HAP)
- Index 14: Stereo Mix (Realtek HD Audio Stereo input)
- Index 15: Microphone Array (Realtek HD Audio Mic Array input)
- Index 18: PC Speaker (Realtek HD Audio output with HAP)
- Index 19: Headset (AMD Bluetooth Audio Device Microphone)
- Index 22: Input (AMD Bluetooth Audio Device Speaker)

In [5]:
# First, install required packages if you haven't:
# pip install pyaudio numpy

import pyaudio
import wave
import numpy as np
import whisper
import threading
import queue
import time

# Initialize audio parameters
CHUNK = 1024 * 3
FORMAT = pyaudio.paFloat32
CHANNELS = 1
RATE = 16000  # Whisper expects 16kHz

# Initialize Whisper model
model = whisper.load_model("base")

# Create a queue for audio chunks
audio_queue = queue.Queue()
is_recording = True

def list_audio_devices():
    p = pyaudio.PyAudio()
    info = []
    print("\nAvailable Audio Devices:")
    for i in range(p.get_device_count()):
        dev_info = p.get_device_info_by_index(i)
        if dev_info.get('maxInputChannels') > 0:  # Only input devices
            print(f"Index {i}: {dev_info.get('name')}")
            info.append(dev_info)
    p.terminate()
    return info

def record_audio():
    p = pyaudio.PyAudio()
    
    # Get device info
    device_index = 1  # Try Realtek Mic first
    try:
        device_info = p.get_device_info_by_index(device_index)
        print(f"Using device: {device_info['name']}")
        
        # Verify device parameters
        supported_rate = int(device_info['defaultSampleRate'])
        channels = min(device_info['maxInputChannels'], CHANNELS)
        
        stream = p.open(format=FORMAT,
                       channels=channels,
                       rate=RATE,  # Force 16kHz sampling rate
                       input=True,
                       input_device_index=device_index,
                       frames_per_buffer=CHUNK)
        
        print("* Recording started...")
        
        while is_recording:
            try:
                data = np.frombuffer(stream.read(CHUNK, exception_on_overflow=False), dtype=np.float32)
                audio_queue.put(data)
            except Exception as e:
                print(f"Error reading from stream: {e}")
                break
                
    except Exception as e:
        print(f"Error setting up audio stream: {e}")
        # Try fallback to default input device
        try:
            stream = p.open(format=FORMAT,
                          channels=CHANNELS,
                          rate=RATE,
                          input=True,
                          frames_per_buffer=CHUNK)
            print("* Recording started with default device...")
            
            while is_recording:
                data = np.frombuffer(stream.read(CHUNK, exception_on_overflow=False), dtype=np.float32)
                audio_queue.put(data)
                
        except Exception as e:
            print(f"Error with fallback device: {e}")
    
    finally:
        try:
            stream.stop_stream()
            stream.close()
        except:
            pass
        p.terminate()

def process_audio():
    accumulated_audio = np.array([], dtype=np.float32)
    
    while is_recording:
        # Process audio in ~5-second chunks
        while len(accumulated_audio) < RATE * 5 and is_recording:
            if not audio_queue.empty():
                chunk = audio_queue.get()
                accumulated_audio = np.concatenate((accumulated_audio, chunk))
        
        if len(accumulated_audio) > 0:
            # Save temporary WAV file
            temp_file = "temp_audio.wav"
            with wave.open(temp_file, 'wb') as wf:
                wf.setnchannels(CHANNELS)
                wf.setsampwidth(2)  # 16-bit
                wf.setframerate(RATE)
                wf.writeframes((accumulated_audio * 32767).astype(np.int16).tobytes())
            
            # Transcribe
            result = model.transcribe(temp_file)
            print(f"\rTranscription: {result['text']}", end='', flush=True)
            
            # Reset accumulated audio
            accumulated_audio = np.array([], dtype=np.float32)

list_audio_devices()

# Start recording and processing threads
record_thread = threading.Thread(target=record_audio)
process_thread = threading.Thread(target=process_audio)

record_thread.start()
process_thread.start()

# Let it run for some time (e.g., 30 seconds)
time.sleep(30)

# Stop recording
is_recording = False
record_thread.join()
process_thread.join()


Available Audio Devices:
Index 0: Microsoft Sound Mapper - Input
Index 1: Microphone Array (Realtek(R) Au
Index 4: Primary Sound Capture Driver
Index 5: Microphone Array (Realtek(R) Audio)
Index 9: Microphone Array (Realtek(R) Audio)
Index 10: Microphone (Realtek HD Audio Mic input)
Index 13: PC Speaker (Realtek HD Audio 2nd output with HAP)
Index 14: Stereo Mix (Realtek HD Audio Stereo input)
Index 15: Microphone Array (Realtek HD Audio Mic Array input)
Index 18: PC Speaker (Realtek HD Audio output with HAP)
Index 19: Headset (AMD Bluetooth Audio Device Microphone)
Index 22: Input (AMD Bluetooth Audio Device Speaker)
Using device: Microphone Array (Realtek(R) Au
* Recording started...


c:\Users\kalin\miniconda3\envs\gaiaenv\lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcription:  Can you do real-time streaming? Hello? Do you hear me?